In [10]:
import create_example
from svdunfold import SVDunfold
import svdunfold
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import curve_fit
from scipy.signal import fftconvolve
from matplotlib.colors import LogNorm
from scipy import stats
import random
from scipy.interpolate import interp1d

In [11]:
nevents=10000
nevents1=100000

x0 = 1836.4 # M2
gamma0 = 8.2
x1 =  2047 # M1
gamma1 = 14.49
x2 =  411.71 # M1
gamma2 = 20.
mu = 0
N_detectors = 100
sigma_0 = 5
sigma_1 = 7
x_min=0.
x_max=2600
x_nbins=130
b_min=0.
b_max=2600
b_nbins=130
type_unf= "SVD"
d0 = 10
dd0 = 20
d1 = 10
dd1 = 20
d2 = 10
dd2 = 20

cauchy_par_test_0=[x0,gamma0]
cauchy_par_test_1=[x1,gamma1]
cauchy_par_test_2=[x2,gamma2]

In [12]:
example_SVD = create_example.CreateExample(N_detectors, nevents, nevents1,x_min, x_max,x_nbins,b_min,b_max,b_nbins, type_unf)
example_SVD_sys_p = create_example.CreateExample(N_detectors, nevents, nevents1,x_min, x_max,x_nbins,b_min,b_max,b_nbins, type_unf)
example_SVD_sys_m = create_example.CreateExample(N_detectors, nevents, nevents1,x_min, x_max,x_nbins,b_min,b_max,b_nbins, type_unf)

In [13]:
example_SVD.start_x_test_gen(x0,gamma0,x1, gamma1,x2, gamma2)
example_SVD.start_x_ini_gen(d0, dd0, d1, dd1, d2, dd2)
example_SVD.start_b(mu, sigma_0, sigma_1)

In [14]:
example_SVD_sys_p.start_x_test_gen(x0,gamma0,x1, gamma1,x2, gamma2)
example_SVD_sys_p.start_x_ini_gen(d0+10, dd0, d1+10, dd1, d2+10, dd2)
example_SVD_sys_p.start_b(mu, sigma_0, sigma_1)

In [15]:
example_SVD_sys_m.start_x_test_gen(x0,gamma0,x1, gamma1,x2, gamma2)
example_SVD_sys_m.start_x_ini_gen(d0-10, dd0, d1-10, dd1, d2-10, dd2)
example_SVD_sys_m.start_b(mu, sigma_0, sigma_1)

In [16]:
example_SVD.processing()
x_ini = example_SVD.get_x_ini()
B = example_SVD.get_B()
A_matrix = example_SVD.get_A_matrix()
b_test = example_SVD.get_b_test()
x_test = example_SVD.get_x_test()

example_SVD_sys_p.processing()
x_ini_sys_p = example_SVD_sys_p.get_x_ini()
B_sys_p = example_SVD_sys_p.get_B()
A_matrix_sys_p = example_SVD_sys_p.get_A_matrix()
b_test_sys_p = example_SVD_sys_p.get_b_test()
x_test_sys_p = example_SVD_sys_p.get_x_test()

example_SVD_sys_m.processing()
x_ini_sys_m = example_SVD_sys_m.get_x_ini()
B_sys_m = example_SVD_sys_m.get_B()
A_matrix_sys_m = example_SVD_sys_m.get_A_matrix()
b_test_sys_m = example_SVD_sys_m.get_b_test()
x_test_sys_m = example_SVD_sys_m.get_x_test()

In [17]:
unfold = []

for i in range(N_detectors):
    unfold.append(svdunfold.SVDunfold(x_ini[i], b_test[i], A_matrix[i], B[i], x_min,x_max,b_min,b_max, 0.001))
    unfold[i].pre_unfolding()

chi2 = []
kk = []
min_k = np.full(N_detectors, -1)
minn = np.full(N_detectors, np.inf)

for k in range(x_nbins):
    for i in range(N_detectors):
        unfold[i].unfolding(k)
        kk.append(k)
        x = unfold[i].get_x_unfolded()
        X = unfold[i].get_X_unfolded_covariance()
        a = np.sum(((x - x_test[i])**2) / np.sqrt(np.diag(X)))
        chi2.append(a)
        
        if minn[i] > a:
            minn[i] = a
            min_k[i] = k
for i in range(N_detectors):
    if min_k[i]>80:
        print(f"Controlla N detector {i} perche ce {min_k[i]}")
        min_k[i]=50
print(min_k)

[54 58 57 58 64 59 59 61 59 58 57 62 59 59 57 60 64 53 51 62 62 57 63 57
 57 51 62 57 60 55 58 56 62 59 52 60 56 61 62 50 57 62 57 62 59 59 58 57
 58 60 57 60 58 61 58 56 57 60 56 58 64 61 57 58 57 55 63 61 51 57 61 60
 62 57 59 64 56 50 58 57 58 64 59 57 60 55 51 55 58 64 52 56 58 60 60 58
 52 57 60 58]


In [ ]:
unfold_sys_p = []

for i in range(N_detectors):
    unfold_sys_p.append(svdunfold.SVDunfold(x_ini_sys_p[i], b_test_sys_p[i], A_matrix_sys_p[i], B_sys_p[i], x_min,x_max,b_min,b_max, 0.001))
    unfold_sys_p[i].pre_unfolding()

chi2_sys_p = []
kk_sys_p = []
min_k_sys_p = np.full(N_detectors, -1)
minn_sys_p = np.full(N_detectors, np.inf)

for k in range(x_nbins):
    for i in range(N_detectors):
        unfold_sys_p[i].unfolding(k)
        kk_sys_p.append(k)
        x_sys_p = unfold_sys_p[i].get_x_unfolded()
        X_sys_p = unfold_sys_p[i].get_X_unfolded_covariance()
        a_sys_p = np.sum(((x_sys_p - x_test_sys_p[i])**2) / np.sqrt(np.diag(X_sys_p)))
        chi2_sys_p.append(a_sys_p)
        
        if minn_sys_p[i] > a_sys_p:
            minn_sys_p[i] = a_sys_p
            min_k_sys_p[i] = k
for i in range(N_detectors):
    if min_k_sys_p[i]>80:
        print(f"Controlla N detector {i} perche ce {min_k_sys_p[i]}")
        min_k_sys_p[i]=50
print(min_k_sys_p)

In [ ]:
unfold_sys_m = []

for i in range(N_detectors):
    unfold_sys_m.append(svdunfold.SVDunfold(x_ini_sys_m[i], b_test_sys_m[i], A_matrix_sys_m[i], B_sys_m[i], x_min,x_max,b_min,b_max, 0.001))
    unfold_sys_m[i].pre_unfolding()

chi2_sys_m = []
kk_sys_m = []
min_k_sys_m = np.full(N_detectors, -1)
minn_sys_m = np.full(N_detectors, np.inf)

for k in range(x_nbins):
    for i in range(N_detectors):
        unfold_sys_m[i].unfolding(k)
        kk_sys_m.append(k)
        x_sys_m = unfold_sys_m[i].get_x_unfolded()
        X_sys_m = unfold_sys_m[i].get_X_unfolded_covariance()
        a_sys_m = np.sum(((x_sys_m - x_test_sys_m[i])**2) / np.sqrt(np.diag(X_sys_m)))
        chi2_sys_m.append(a_sys_m)
        
        if minn_sys_m[i] > a_sys_m:
            minn_sys_m[i] = a_sys_m
            min_k_sys_m[i] = k
for i in range(N_detectors):
    if min_k_sys_m[i]>80:
        print(f"Controlla N detector {i} perche ce {min_k_sys_m[i]}")
        min_k_sys_m[i]=50
print(min_k_sys_m)

In [ ]:
d=[]
for i in range(N_detectors):
    d.append(unfold[i].get_abs_d())

plt.figure(figsize=(10, 7))

plt.plot(np.arange(len(d[0])), np.ones(len(d[0])), "r-" )
plt.plot(np.arange(len(d[0])), d[0], "k.--")
plt.xlabel("i")
plt.ylabel("|d|")
plt.yscale("log")

In [ ]:
x = []
err = []
X = []
for i in range(N_detectors):
    unfold[i].unfolding(min_k[i])  # Chiama il metodo unfolding
    x.append(unfold[i].get_x_unfolded())
    X.append(unfold[i].get_X_unfolded_covariance())
    err.append(np.sqrt(np.diag(X[i])))

x_sys_p = []
err_sys_p = []
X_sys_p = []
for i in range(N_detectors):
    unfold_sys_p[i].unfolding(min_k_sys_p[i])  # Chiama il metodo unfolding
    x_sys_p.append(unfold_sys_p[i].get_x_unfolded())
    X_sys_p.append(unfold_sys_p[i].get_X_unfolded_covariance())
    err_sys_p.append(np.sqrt(np.diag(X_sys_p[i])))

x_sys_m = []
err_sys_m = []
X_sys_m = []
for i in range(N_detectors):
    unfold_sys_m[i].unfolding(min_k_sys_m[i])  # Chiama il metodo unfolding
    x_sys_m.append(unfold_sys_m[i].get_x_unfolded())
    X_sys_m.append(unfold_sys_m[i].get_X_unfolded_covariance())
    err_sys_m.append(np.sqrt(np.diag(X_sys_m[i])))

In [ ]:
x = np.array(x, dtype=float)
err = np.array(err, dtype=float)
N_d = np.linspace(0, N_detectors, N_detectors)

x_sys_p = np.array(x_sys_p, dtype=float)
err_sys_p = np.array(err_sys_p, dtype=float)

x_sys_m = np.array(x_sys_m, dtype=float)
err_sys_m = np.array(err_sys_m, dtype=float)


In [ ]:
bin_centers_x = example_SVD.get_bin_centers_x()
b_final_test = np.mean(b_test, axis=0)
x_final_test_1 = stats.cauchy.pdf(bin_centers_x,cauchy_par_test_0[0], cauchy_par_test_0[1])
x_final_test_2 = stats.cauchy.pdf(bin_centers_x,cauchy_par_test_1[0], cauchy_par_test_1[1])
x_final_test_3 = stats.cauchy.pdf(bin_centers_x,cauchy_par_test_2[0], cauchy_par_test_2[1])

x_final_test = (x_final_test_2+x_final_test_1+x_final_test_3)  
scale_f = np.mean(x_test, axis=0)/ x_final_test
x_final_test = x_final_test * scale_f

In [ ]:
weights = 1 / (err ** 2)
denominator = np.sum(weights, axis=0)
numerator = np.sum((x * weights), axis=0)
xxx = numerator / denominator
errr = np.sqrt(1./denominator)

weights_sys_p = 1 / (err_sys_p ** 2)
denominator_sys_p = np.sum(weights_sys_p, axis=0)
numerator_sys_p = np.sum((x_sys_p * weights_sys_p), axis=0)
xxx_sys_p = numerator_sys_p / denominator_sys_p
errr_sys_p = np.sqrt(1./denominator_sys_p)

weights_sys_m = 1 / (err_sys_m ** 2)
denominator_sys_m = np.sum(weights_sys_m, axis=0)
numerator_sys_m = np.sum((x_sys_m * weights_sys_m), axis=0)
xxx_sys_m = numerator_sys_m / denominator_sys_m
errr_sys_m = np.sqrt(1./denominator_sys_m)

In [ ]:
x_final=np.median(x, axis=0)
error_tot = np.sqrt(stats.median_abs_deviation(x, axis=0)**2)

x_final_sys_p=np.median(x_sys_p, axis=0)
error_tot_sys_p = np.sqrt(stats.median_abs_deviation(x_sys_p, axis=0)**2)

x_final_sys_m=np.median(x_sys_m, axis=0)
error_tot_sys_m = np.sqrt(stats.median_abs_deviation(x_sys_m, axis=0)**2)

error_tot = np.sqrt((x_final_sys_p-x_final_sys_m)**2 + error_tot**2)

In [ ]:
plt.figure(2)
plt.errorbar(bin_centers_x,x_final_test, marker=".", yerr=error_tot, linestyle="")
plt.errorbar(bin_centers_x,x_final, marker=".", yerr=error_tot, linestyle="")
plt.yscale("log")
plt.figure(66)
plt.axvline(x=x_final_test[102], color='b', label='true value')
plt.errorbar(x[:,102],N_d, xerr=err[:,102], linestyle="", marker="o", color="black", label="stat")
plt.errorbar(x_final[102],108, xerr=error_tot[102], linestyle="", marker="o", color="red", label="N detectors (median)")
plt.errorbar(xxx[102],112, xerr=errr[102], linestyle="", marker="o", color="blue", label="N detectors (weighted mean)")

plt.legend(loc="lower right")
plt.xlabel("5th $x_{unfolded}$ bin content")
plt.ylabel("#detector")
plt.savefig("/home/sara/Downloads/svdunfold_5th_bin.png")
plt.figure(67)
plt.axvline(x=x_final_test[50], color='b', label='true value')
plt.errorbar(x[:,50],N_d, xerr=err[:,50], linestyle="", marker="o", color="black", label="stat")
plt.errorbar(x_final[50],108, xerr=error_tot[50], linestyle="", marker="o", color="red", label="N detectors (median)")
plt.errorbar(xxx[50],112, xerr=errr[50], linestyle="", marker="o", color="blue", label="N detectors (weighted mean)")

plt.xlabel("50th $x_{unfolded}$ bin content")
plt.ylabel("#detector")
plt.legend(loc="lower right")
plt.savefig("/home/sara/Downloads/svdunfold_50th_bin.png")
plt.figure(68)
d=39
plt.axvline(x=x_final_test[d], color='b', label='true value')
plt.errorbar(x[:,d],N_d, xerr=err[:,d], linestyle="", marker="o", color="black", label="stat")
plt.errorbar(x_final[d],108, xerr=error_tot[d], linestyle="", marker="o", color="red", label="N detectors (median)")
plt.errorbar(xxx[d],112, xerr=errr[d], linestyle="", marker="o", color="blue", label="N detectors (weighted mean)")

plt.xlabel("39th $x_{unfolded}$ bin content")
plt.ylabel("#detector")
plt.legend(loc="lower right")
plt.savefig("/home/sara/Downloads/svdunfold_39th_bin.png")
plt.figure(69)
plt.hist(x[:,d], range=[np.min(x[:,d]),np.max(x[:,d])], bins=20,label="bin 39")
plt.axvline(x=x_final_test[d], color='b', label='true value')
plt.axvline(x=xxx[d], color='g', label='media')
plt.axvline(x=x_final[d], color='r', label='mediana')
plt.legend(loc="lower right")
plt.figure(79)
d=50
plt.hist(x[:,d], range=[np.min(x[:,d]),np.max(x[:,d])], bins=20,label="bin 50")
plt.axvline(x=x_final_test[d], color='b', label='true value')
plt.axvline(x=xxx[d], color='g', label='media')
plt.axvline(x=x_final[d], color='r', label='mediana')
plt.legend(loc="lower right")

plt.figure(89)
d=102
plt.hist(x[:,d], range=[np.min(x[:,d]),np.max(x[:,d])], bins=20,label="bin 102")
plt.axvline(x=x_final_test[d], color='b', label='true value')
plt.axvline(x=xxx[d], color='g', label='media')
plt.axvline(x=x_final[d], color='r', label='mediana')
plt.legend(loc="lower right")

In [ ]:
plt.subplots(2 , 1, figsize=(15, 10))
plt.subplot(2,1,1)
plt.plot(bin_centers_x, x_final_test, ".", color="blue", label="True spectrum ($x_{test}$)")
plt.plot(bin_centers_x, b_final_test, ".", color="red", label="True measured spectrum ($b_{test}$)")
plt.title("SVD unfold")

plt.errorbar(bin_centers_x, x_final, yerr=error_tot, label="$x_{unfolded}$" , marker='.', linestyle="", color="black")
plt.xlabel("Energy [eV]")
plt.ylabel("counts/{} eV".format((x_max-x_min)/x_nbins))
plt.legend()
plt.yscale("log")
plt.subplot(2,1,2)
plt.plot(bin_centers_x,(x_final_test/x_final), marker='.', linestyle="", color="black")
plt.xlabel("Energy [eV]")
plt.ylabel("Data/Reco")
plt.axhline(y=1, color="red")
plt.savefig("/home/sara/Downloads/svdunfold_20detector.png")

In [ ]:
plt.figure(25)
plt.title("SVD unfold")

plt.xlabel("Energy [eV]")
plt.plot(bin_centers_x,(x_final_test-x_final)/error_tot, marker='.', linestyle="", color="black")
plt.xlabel("Energy [eV]")
plt.ylabel("Norm. res.")
plt.axhline(y=0, color="red")
plt.savefig("/home/sara/Downloads/svdunfold_bias_20dec.png")
plt.show()